# 바텀 라인의 영향력 분석

# 1. 환경 준비

In [1]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import warnings

# warnings.filterwarnings(action='ignore')
# %config InlineBackend.figure_format='retina'

In [ ]:
# 데이터 불러오기
data = pd.read_excel('match_log_data.xlsx')
data.head(2)

# 2. 데이터 이해

In [ ]:
# 데이터 확인
data.info()

In [ ]:
# 승패 비율 확인
data['win'].value_counts()

In [ ]:
# 승리 데이터 요약

In [ ]:
# 패배 데이터 요약

In [ ]:
# 결측치 확인
data.isnull().sum()

In [ ]:
# 상관관계 시각화
plt.figure(figsize=(15,15))
sns.heatmap(data=data.corr(),
            annot=True,
            fmt='.2f',
            linewidths=.5,
            cmap='Blues')

# 3. 데이터 준비

In [ ]:
# 승패 True False -> 1, 0
data = data.replace({'win': {True:1}}, {'win': {False:0}})
data.head()

In [ ]:
# 변수 제거
drop_cols = ['puuid', 'matchid']

data.drop(drop_cols, axis=1, inplace=True)

In [ ]:
# x, y 분리
target = 'win'

x = data.drop(target, axis=1)
y = data.loc[:, target]

In [ ]:
# 가변수화
dumm_cols = ['level15']

x = pd.get_dummies(x, columns=dumm_cols, drop_first=True)
x.head()

In [2]:
# train, val, test 분리
from sklearn.model_selection import train_test_split

# train:test = 8:2
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

# # train:val:test = 7:2:1
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)
# x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.33, random_state=1)


In [ ]:
# 정규화
from sklearn.preprocessing import MinMaxScaler

# 정규화
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

# 4. 모델링

In [ ]:
# 1) 불러오기
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# 2) 선언하기
model = DecisionTreeClassifier(max_depth=10, random_state=1)

In [ ]:
# 3) 학습하기
model.fit(x_train, y_train)

In [ ]:
# 4) 예측하기
y_pred = model.predict(x_test)

In [ ]:
# 5) 평가하기
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# 5. 시각화 및 변수 중요도

In [ ]:
# 시각화 모듈 불러오기
from sklearn.tree import export_graphviz
from IPython.display import Image

# 이미지 파일 만들기
export_graphviz(model,                                 # 모델 이름
                out_file='tree.dot',                   # 파일 이름 
                feature_names=x.columns,               # Feature 이름
                class_names=['lose', 'win'],           # Target Class 이름
                rounded=True,                          # 둥근 테두리
                precision=2,                           # 불순도 소숫점 자리수
                max_depth=3,                           # 출력할 트리 깊이
                filled=True)                           # 박스 내부 채우기

# 파일 변환
!dot tree.dot -Tpng -otree.png -Gdpi=300

# 이미지 파일 표시
Image(filename='tree.png')

In [ ]:
# 변수 중요도
plt.figure(figsize=(10, 10))
plt.barh(list(x), model.feature_importances_)
plt.show()

In [ ]:
# 변수 중요도 정렬
perf_dic = {'feature':list(x), 'importance': model.feature_importances_}
df = pd.DataFrame(perf_dic)
df.sort_values(by='importance', ascending=True, inplace=True)

# 시각화
plt.figure(figsize=(10, 10))
plt.barh(df['feature'], df['importance'])
plt.show()

# 6. 인사이트

* ~ 데이터가 중요함
* 실제로는 영향력이 ~